Exploración inicial del dataset LANL  o EDA (Exploratory Data Analysis)

En este primer notebook se va a realizar un análisis exploratorio inicial del dataset de autenticación de Los Alamos National Laboratory (LANL).
El objetivo de esta primera fase es comprender la estructura y validar su idoneidad para la detección de anomalías en entornos IAM.

In [33]:
import pandas as pd
import numpy as np

In [34]:
#En este aprtado se analiza primeramente el tamaño del dataset
import os

file_size_gb = os.path.getsize("auth.txt") / (1024**3)
print(f"Tamaño del dataset: {file_size_gb:.2f} GB")

Tamaño del dataset: 68.37 GB


Dado el elevado tamaño del dataset original, la exploración inicial se realiza sobre una muestra representativa de los datos. 
De esta manera se va a poder analizar la estructura y características principales del conjunto de datos sin comprometer el rendimiento del sistema.

A continuación, vamos a definir el nombre de las cabeceras para diferenciar los tipos de datos que hay en la BBDD.
Para poder distinguir cada una de las columnas, se ha seguido la información proporcionada por la página web donde se encuentra el dataset.

In [35]:
#Definición del nombre de las cabeceras
columns = [
    "time",
    "src_user",
    "dst_user",
    "src_host",
    "dst_host",
    "auth_type",
    "logon_type",
    "auth_orientation",
    "auth_result"
]

Se realiza una primera carga parcial del dataset.

In [36]:
#Se cargan un total de 1000000 filas iniciales para poder distinguir el tipo de información que aparece.
df = pd.read_csv(
    "auth.txt",
    names=columns,
    nrows=1_000_000
)

df.head()

,time,src_user,dst_user,src_host,dst_host,auth_type,logon_type,auth_orientation,auth_result
0,1,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C1250,C586,NTLM,Network,LogOn,Success
1,1,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success
2,1,C101$@DOM1,C101$@DOM1,C988,C988,?,Network,LogOff,Success
3,1,C1020$@DOM1,SYSTEM@C1020,C1020,C1020,Negotiate,Service,LogOn,Success
4,1,C1021$@DOM1,C1021$@DOM1,C1021,C625,Kerberos,Network,LogOn,Success


In [37]:
df

,time,src_user,dst_user,src_host,dst_host,auth_type,logon_type,auth_orientation,auth_result
0,1,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C1250,C586,NTLM,Network,LogOn,Success
1,1,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success
2,1,C101$@DOM1,C101$@DOM1,C988,C988,?,Network,LogOff,Success
3,1,C1020$@DOM1,SYSTEM@C1020,C1020,C1020,Negotiate,Service,LogOn,Success
4,1,C1021$@DOM1,C1021$@DOM1,C1021,C625,Kerberos,Network,LogOn,Success
...,...,...,...,...,...,...,...,...,...
999995,10208,U19@DOM1,U19@DOM1,C229,C229,?,Network,LogOff,Success
999996,10208,U207@DOM1,U207@DOM1,C529,C529,?,Network,LogOff,Success
999997,10208,U22@DOM1,U22@DOM1,C452,C457,Kerberos,Network,LogOn,Success
999998,10208,U22@DOM1,U22@DOM1,C457,C457,?,Network,LogOff,Success


Ahora que se ha cargado parcialmente los datos del dataset y conocemos las columnas en las que se divide, se realizará un analisis del comportamiento de los datos.

ESTRUCTURA Y TIPOS DE DATOS

In [38]:
#Comprobación del numero de filas, columnas, tipos de datos en cada columna, la cantidad de valores nulos y el uso de memoria total.
df.info()
df.shape

<class 'pandas.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column            Non-Null Count    Dtype
---  ------            --------------    -----
 0   time              1000000 non-null  int64
 1   src_user          1000000 non-null  str  
 2   dst_user          1000000 non-null  str  
 3   src_host          1000000 non-null  str  
 4   dst_host          1000000 non-null  str  
 5   auth_type         1000000 non-null  str  
 6   logon_type        1000000 non-null  str  
 7   auth_orientation  1000000 non-null  str  
 8   auth_result       1000000 non-null  str  
dtypes: int64(1), str(8)
memory usage: 68.7 MB


(1000000, 9)

ANÁLISIS ESTADÍSTICO DESCRIPTIVO

In [39]:
df.describe(include="all")

,time,src_user,dst_user,src_host,dst_host,auth_type,logon_type,auth_orientation,auth_result
count,1000000.000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000
unique,NaN,7052,8822,4085,4094,12,10,5,2
top,NaN,U22@DOM1,U22@DOM1,C586,C586,?,Network,LogOff,Success
freq,NaN,27855,27855,65302,133651,583802,827731,430915,993123
mean,5101.500575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2955.762085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2545.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,5118.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,7674.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


DISTRIBUCIÓN DE RESULTADOS DE AUTENTICACIÓN

In [40]:
df["auth_result"].value_counts()

auth_result
Success    993123
Fail         6877
Name: count, dtype: int64

In [41]:
6877/993123

0.006924620615976067

CONVERSIÓN TEMPORAL

In [43]:
df["time"].min(), df["time"].max()

(np.int64(1), np.int64(10208))

In [46]:
#Transformación de la columna Time para un mejor analísis
df["time"] = pd.to_datetime(df["time"], unit="s")
df.set_index("time", inplace=True)
#df["time"].head()

df.index.min(), df.index.max()

(Timestamp('1970-01-01 00:00:01'), Timestamp('1970-01-01 02:50:08'))

In [45]:
df

,time,src_user,dst_user,src_host,dst_host,auth_type,logon_type,auth_orientation,auth_result
0,1970-01-01 00:00:01,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C1250,C586,NTLM,Network,LogOn,Success
1,1970-01-01 00:00:01,ANONYMOUS LOGON@C586,ANONYMOUS LOGON@C586,C586,C586,?,Network,LogOff,Success
2,1970-01-01 00:00:01,C101$@DOM1,C101$@DOM1,C988,C988,?,Network,LogOff,Success
3,1970-01-01 00:00:01,C1020$@DOM1,SYSTEM@C1020,C1020,C1020,Negotiate,Service,LogOn,Success
4,1970-01-01 00:00:01,C1021$@DOM1,C1021$@DOM1,C1021,C625,Kerberos,Network,LogOn,Success
...,...,...,...,...,...,...,...,...,...
999995,1970-01-01 02:50:08,U19@DOM1,U19@DOM1,C229,C229,?,Network,LogOff,Success
999996,1970-01-01 02:50:08,U207@DOM1,U207@DOM1,C529,C529,?,Network,LogOff,Success
999997,1970-01-01 02:50:08,U22@DOM1,U22@DOM1,C452,C457,Kerberos,Network,LogOn,Success
999998,1970-01-01 02:50:08,U22@DOM1,U22@DOM1,C457,C457,?,Network,LogOff,Success


AUTHENTICATION TYPE

In [49]:
df["auth_type"].unique()

<StringArray>
[                                 'NTLM',
                                     '?',
                             'Negotiate',
                              'Kerberos',
 'MICROSOFT_AUTHENTICATION_PACKAGE_V1_0',
          'MICROSOFT_AUTHENTICATION_PAC',
      'MICROSOFT_AUTHENTICATION_PACKAGE',
       'MICROSOFT_AUTHENTICATION_PACKAG',
   'MICROSOFT_AUTHENTICATION_PACKAGE_V1',
     'MICROSOFT_AUTHENTICATION_PACKAGE_',
           'MICROSOFT_AUTHENTICATION_PA',
         'MICROSOFT_AUTHENTICATION_PACK']
Length: 12, dtype: str

LOGON TYPE

In [50]:
df["logon_type"].unique()

<StringArray>
[          'Network',           'Service',             'Batch',
                 '?',       'Interactive',  'NetworkCleartext',
    'NewCredentials',            'Unlock', 'RemoteInteractive',
 'CachedInteractive']
Length: 10, dtype: str

AUTHENTICATION ORIENTATION

In [51]:
df["auth_orientation"].unique()

<StringArray>
['LogOn', 'LogOff', 'TGS', 'AuthMap', 'TGT']
Length: 5, dtype: str

In [65]:
df["src_host"].unique()

<StringArray>
['C1250',  'C586',  'C988', 'C1020', 'C1021', 'C1035', 'C1069', 'C1085',
  'C612', 'C1151',
 ...
 'C5659', 'C5791', 'C5674', 'C5698', 'C5929', 'C5681', 'C5931', 'C5707',
 'C5801', 'C5690']
Length: 4085, dtype: str

In [67]:
df.groupby(["auth_type", "auth_result"]).size()

auth_type                              auth_result
?                                      Fail             5311
                                       Success        578491
Kerberos                               Fail               37
                                       Success        359905
MICROSOFT_AUTHENTICATION_PA            Success             1
MICROSOFT_AUTHENTICATION_PAC           Success             3
MICROSOFT_AUTHENTICATION_PACK          Success             1
MICROSOFT_AUTHENTICATION_PACKAG        Fail                3
                                       Success            59
MICROSOFT_AUTHENTICATION_PACKAGE       Success            38
MICROSOFT_AUTHENTICATION_PACKAGE_      Success             7
MICROSOFT_AUTHENTICATION_PACKAGE_V1    Success            17
MICROSOFT_AUTHENTICATION_PACKAGE_V1_0  Fail              482
                                       Success           162
NTLM                                   Fail              688
                                  

In [66]:
df.groupby(["logon_type", "auth_result"]).size()

logon_type         auth_result
?                  Fail             5311
                   Success        147576
Batch              Fail              272
                   Success          3324
CachedInteractive  Fail                3
                   Success             6
Interactive        Fail               55
                   Success          5058
Network            Fail             1208
                   Success        826523
NetworkCleartext   Success           841
NewCredentials     Fail               17
                   Success           140
RemoteInteractive  Fail                8
                   Success            23
Service            Success          9587
Unlock             Fail                3
                   Success            45
dtype: int64

En estos dos últimos caso, se intenta comparar cuales fallan y cuales han salido exitoso para identificar posibles superficies de ataque y clarificar el resultado final.

A partir de este primer análisis exploratorio se van a identificar posibles indicadores de comportamiento anómalo.

src_user
C3583$@DOM1    2
C2653$@DOM1    2
C1085$@DOM1    2
C1149$@DOM1    2
C1846$@DOM1    2
C2631$@DOM1    2
C2654$@DOM1    2
C2676$@DOM1    2
C3677$@DOM1    2
C5426$@DOM1    2
Name: auth_result, dtype: int64

In [53]:
#En este primer punto, se van a identificar usuarios (src_user) que acceden a un número elevado de sistemas diferentes (dst_host).
df.groupby("src_user")["dst_host"].nunique().sort_values(ascending=False).head (10)

src_user
U66@DOM1      127
U78@DOM1       35
U292@DOM1      33
U24@DOM1       30
U491@DOM1      29
C523$@DOM1     25
U881@DOM1      24
U179@DOM1      23
U1055@DOM1     22
U1244@DOM1     22
Name: dst_host, dtype: int64

In [ ]:
#Ratio de accesos fallidos por usuario, mostrandolo los 10 primeros de la lista
is_fail = df["auth_result"].astype(str).str.lower() == "fail"

fail_ratio_user = (
    df.assign(is_fail=is_fail)
      .groupby("src_user")["is_fail"]
      .mean()
      .sort_values(ascending=False)
)

fail_ratio_user.head(10)


src_user
C5167$@?       1.0
U1844@?        1.0
C1527$@DOM1    1.0
U1851@?        1.0
C4858$@DOM1    1.0
U1855@?        1.0
U199@?         1.0
C3580$@DOM1    1.0
U205@C2625     1.0
C3131$@DOM1    1.0
Name: is_fail, dtype: float64

In [ ]:
#Usuarios con gran actividad de acceso
attempts_user = df["src_user"].value_counts()
attempts_user.head(10)

src_user
U22@DOM1                27855
U66@DOM1                20409
C599$@DOM1              19376
ANONYMOUS LOGON@C586    17934
C1114$@DOM1             15141
C585$@DOM1              14013
C104$@DOM1              12920
C743$@DOM1              12101
C567$@DOM1              10281
C123$@DOM1               9913
Name: count, dtype: int64

In [ ]:
#Acceso de usuarios en horarios inusuales, como por ejemplo de noche.
df["hour"] = df.index.hour
night_access = df["hour"].between(0, 6)

night_ratio_user = (
    df.assign(night=night_access)
      .groupby("src_user")["night"]
      .mean()
      .sort_values(ascending=False)
)

night_ratio_user.head(10)


src_user
ANONYMOUS LOGON@C1042    1.0
ANONYMOUS LOGON@C1065    1.0
ANONYMOUS LOGON@C1089    1.0
ANONYMOUS LOGON@C1139    1.0
ANONYMOUS LOGON@C1208    1.0
ANONYMOUS LOGON@C135     1.0
ANONYMOUS LOGON@C1503    1.0
ANONYMOUS LOGON@C1529    1.0
ANONYMOUS LOGON@C1567    1.0
ANONYMOUS LOGON@C1570    1.0
Name: night, dtype: float64

In [ ]:
# Usuarios que han accedido desde diferentes hosts de origen, indicando un posible robo de credenciales.
src_host_diversity = (
    df.groupby("src_user")["src_host"]
      .nunique()
      .sort_values(ascending=False)
)

src_host_diversity.head(10)


src_user
ANONYMOUS LOGON@C586     625
ANONYMOUS LOGON@C467     255
ANONYMOUS LOGON@C457     218
ANONYMOUS LOGON@C529     214
ANONYMOUS LOGON@C612     209
ANONYMOUS LOGON@C2106    174
ANONYMOUS LOGON@C1065    150
U66@DOM1                 102
ANONYMOUS LOGON@C625      86
ANONYMOUS LOGON@C528      79
Name: src_host, dtype: int64

In [ ]:
#usuarios que acceden a muchos usuarios destino
user_to_user = (
    df.groupby("src_user")["dst_user"]
      .nunique()
      .sort_values(ascending=False)
)

user_to_user.head(10)


src_user
U20@DOM1       65
U78@DOM1       24
U114@DOM1      10
U24@DOM1       10
U428@DOM1       8
C1570$@DOM1     8
C291$@DOM1      7
C1042$@DOM1     7
C1567$@DOM1     7
C2254$@DOM1     7
Name: dst_user, dtype: int64

In [ ]:
#usuarios que han utilizado diferentes tipos de autenticación
auth_type_user = (
    df.groupby("src_user")["auth_type"]
      .nunique()
      .sort_values(ascending=False)
)

auth_type_user.head(10)


src_user
C586$@DOM1     6
C457$@DOM1     5
C1065$@DOM1    5
C1640$@DOM1    5
C467$@DOM1     5
C528$@DOM1     5
C625$@DOM1     5
C612$@DOM1     5
C2654$@DOM1    5
C365$@DOM1     4
Name: auth_type, dtype: int64

HIPÓTESIS

Como hemos podido ver a lo largo de este notebook, el dataset contiene diferentes tipos de datos cualitativos, excepto por la columna del tiempo, el cual es el único valor númerico de la tabla. Hemos visto que no se encuentran datos nulos, y que los que no contienen un valor válido son representados con una interrogación '?'. 
Al dividir los datos en columnas desde un inicio, se ha podido ver mejor que tipos de datos hay y cuales son sus valores.
Se ha analizado que tipos de inicios de sesion se han utilizado, que tipos de autenticaciones utilizan los usuarios, y además se han estudiado diferentes indicadores de comportamiento posiblemente anómalo.

Se puede ver que el comportamiento normal de los usuarios presenta diferentes patrones relativamente estables en términos de volumen de accesos, horarios dentro de lo habitual y el número de sistemas accedidos. 
Pero también, los accesos atípicos o potencialmente maliciosos se caracterizan por desviaciones significativas respecto a estos patrones, tales como un aumento anómalo de accesos fallidos, actividad fuera de horarios habituales, acceso a un número inusualmente elevado de sistemas distintos o picos de actividad concentrados en intervalos de tiempo reducidos.
